In [ ]:
%load_ext autoreload
%autoreload 2

import spectral_unmixing_tools as el_spectral

## Extract spectra as array

In [1]:
import rasterio
import numpy as np

class ENVIProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None  # This will hold the raster data array
        self.file_type = "envi"
        # Other attributes...

    def load_data(self):
        """Loads the raster data from the file_path into self.data"""
        with rasterio.open(self.file_path) as src:
            self.data = src.read()  # Read all bands

    def get_chunk_from_extent(self, corrections=[], resample=False):
        self.load_data()  # Ensure data is loaded
        with rasterio.open(self.file_path) as src:
            bounds = src.bounds
            width, height = src.width, src.height
            col_start, line_start = 0, 0
            col_end, line_end = width, height

            # Assuming self.data is a 3D numpy array with dimensions [bands, rows, cols]
            chunk = self.data[:, line_start:line_end, col_start:col_end]
            
            # Apply any processing to chunk here...
            # For example, to demonstrate, flip chunk vertically
            chunk = np.flip(chunk, axis=1)
            
            return chunk



# Example usage:
# Replace 'your_envi_file_path' with the actual path to your ENVI file
raster_path = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200807_155314_reflectance/NEON_D13_NIWO_DP1_20200807_155314_reflectanceNEON_D13_NIWO_DP1_20200807_155314_reflectance__envi"  # Update this to your actual raster file path
processor = ENVIProcessor(raster_path)
chunk = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)



In [4]:
## flatten array into a 2D df


In [2]:
import pandas as pd
import numpy as np

def flatten_array_to_dataframe(array):
    """
    Flattens a 3D numpy array into a pandas DataFrame.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Returns:
    - A pandas DataFrame where each row represents a pixel across all bands.
    """
    # Ensure the input is a 3D numpy array
    if len(array.shape) != 3:
        raise ValueError("Input array must be 3-dimensional.")
    
    bands, rows, cols = array.shape
    # Reshape the array to have pixels as rows and bands as columns
    reshaped_array = array.reshape(bands, -1).T  # Transpose to make bands as columns
    
    # Create a DataFrame from the reshaped array
    df = pd.DataFrame(reshaped_array, columns=[f'Band_{i+1}' for i in range(bands)])
    
    # Optionally, add pixel row and column indices
    pixel_indices = np.indices((rows, cols)).reshape(2, -1).T  # 2D array of row,col indices for each pixel
    df['Pixel_Row'] = pixel_indices[:, 0]
    df['Pixel_Col'] = pixel_indices[:, 1]
    
    return df

# Example usage:
# Assuming 'chunk' is your 3D numpy array of shape (426, 11138, 1031)
chunk = np.random.rand(426, 11138, 1031)  # Example array, replace with your actual data
df = flatten_array_to_dataframe(chunk)



In [3]:
df

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,...,Band_419,Band_420,Band_421,Band_422,Band_423,Band_424,Band_425,Band_426,Pixel_Row,Pixel_Col
0,0.797209,0.314943,0.515625,0.289574,0.567998,0.711330,0.985263,0.488050,0.669980,0.047356,...,0.251188,0.565922,0.188862,0.551562,0.465831,0.023405,0.336994,0.450468,0,0
1,0.936276,0.837195,0.695625,0.631541,0.701025,0.960226,0.141791,0.381211,0.114170,0.788448,...,0.230309,0.182223,0.290250,0.975250,0.227430,0.175060,0.567754,0.865518,0,1
2,0.283778,0.107825,0.300096,0.302607,0.572963,0.244061,0.893596,0.930337,0.607978,0.133053,...,0.597720,0.048488,0.564245,0.728381,0.535179,0.961610,0.464103,0.565047,0,2
3,0.457389,0.719769,0.118473,0.407864,0.979559,0.097455,0.728281,0.564473,0.634733,0.200184,...,0.239004,0.365207,0.195417,0.237427,0.699486,0.205661,0.734998,0.589876,0,3
4,0.366526,0.529941,0.417289,0.847633,0.193126,0.982161,0.822412,0.445544,0.044492,0.017709,...,0.742343,0.772829,0.340681,0.146870,0.946994,0.066265,0.499453,0.388927,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11483273,0.152260,0.766855,0.169944,0.878360,0.383142,0.539870,0.985659,0.650074,0.760505,0.952661,...,0.317700,0.078385,0.648448,0.056393,0.945091,0.665414,0.529980,0.593835,11137,1026
11483274,0.345999,0.479799,0.496870,0.479557,0.999687,0.157588,0.562706,0.481475,0.395999,0.912812,...,0.514387,0.211770,0.179397,0.664200,0.767492,0.368559,0.885074,0.608405,11137,1027
11483275,0.748482,0.771680,0.299341,0.296593,0.102600,0.957731,0.038437,0.549724,0.076379,0.128406,...,0.251922,0.673968,0.346867,0.888135,0.471597,0.753125,0.630612,0.409271,11137,1028
11483276,0.526175,0.401877,0.607318,0.477941,0.549137,0.047495,0.769956,0.422864,0.126572,0.073549,...,0.406933,0.387700,0.204090,0.547156,0.558324,0.079974,0.169205,0.287978,11137,1029


## Reshape data frame and change labels

In [ ]:
import numpy as np

def flatten_and_melt_array_to_structured_array(array):
    """
    Flattens a 3D numpy array to a structured numpy array in the "melted" format.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Returns:
    - A structured numpy array with 'Pixel_Row', 'Pixel_Col', 'Band_ID', and 'Wavelength' fields.
    """
    bands, rows, cols = array.shape
    total_pixels = rows * cols
    dtype = [('Pixel_Row', int), ('Pixel_Col', int), ('Band_ID', 'U10'), ('Wavelength', array.dtype)]
    
    # Create an empty structured array
    structured_array = np.zeros(total_pixels * bands, dtype=dtype)
    
    # Populate the structured array
    counter = 0
    for row in range(rows):
        for col in range(cols):
            for band in range(bands):
                structured_array[counter] = (row, col, f'Band_{band+1}', array[band, row, col])
                counter += 1
                
    return structured_array

# Example usage
chunk = np.random.rand(426, 11138, 1031)  # Example array, replace with your actual data
melted_array = flatten_and_melt_array_to_structured_array(chunk)


In [ ]:
melted_array